In [1]:
from IPython.display import display, HTML
display(HTML('<h1><img src="logo.png" style="background-color: black"></h1>'))

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Annual Prediction 

In [3]:
train2 = pd.read_csv("Bihis 2020.csv")

In [4]:
train1 = pd.read_csv("Bihis 2021.csv")

In [5]:
test = pd.read_csv("Bihis 2022.csv")

In [6]:
train2_len = len(train2)
train1_len = len(train1)
test_len = len(test)

In [7]:
df = pd.concat([train2, train1, test], axis=0)
pd.set_option('display.max_rows', df.shape[0]+1)
df.reset_index(drop=True, inplace=True)

In [8]:
%%capture
print(len(train2), len(train1), len(test), len(df))

In [9]:
%%capture
df.apply(lambda x: len(x.unique()))

In [10]:
%%capture
df.isnull().sum()

In [11]:
%%capture
cat_col = []
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
cat_col

In [12]:
%%capture
cat_col.remove('Name')
cat_col

In [13]:
%%capture
for col in cat_col:
    print(col)
    print(df[col].value_counts())
    print();

In [14]:
%%capture
size_mode = df.pivot_table(values = 'Size', columns = 'Category', aggfunc=(lambda x: x.mode()[0]))
size_mode

In [15]:
miss_bool = df['Size'].isnull()
df.loc[miss_bool, 'Size'] = df.loc[miss_bool, 'Category'].apply(lambda x: size_mode[x])

In [16]:
%%capture
df['Size'].isnull().sum()

In [17]:
df['Total_Year'] = 2022 - df['Year']

In [18]:
%%capture
df['Total_Year']

In [19]:
%%capture
df = pd.get_dummies(df, columns=[ 'Size', 'Category'])
df.head();

In [20]:
%%capture
df.tail();

In [21]:
%%capture
train2 = df.iloc[:train2_len,:]
train1 = df.iloc[train2_len:,:]
test = df.iloc[train2_len:,:]

In [22]:
%%capture
test = test.drop(columns=['Sales'])
test.head()

In [23]:
%%capture
test.tail();

In [24]:
X = train2.drop(columns=['Year', 'Name', 'Sales'])
y = train2['Sales']

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
def train2(model, X, y):
    model.fit(X, y)
    model.score(X, y)
    

# Accuracy of model

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
train2(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)

In [27]:
%%capture
x_test = test.drop(columns=['Year', 'Name'])
x_test.head();

# Prediction

In [28]:
pred = model.predict(X)
cv_score = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
cv_score = np.abs(np.mean(cv_score))
print("Model Report Score:", model.score(X, y))
print("Accuracy:", mean_squared_error(y,pred))
print("CrossValidationScore:", cv_score)


Model Report Score: 0.8199548733018005
Accuracy: 756756.4905620199
CrossValidationScore: 5490420.491464824


In [29]:
pred = model.predict(x_test)
pred

array([2726.0952381 , 3782.75095238, 1903.3       , 2640.62238095,
       7645.46571429, 3392.5       , 2239.87833333, 3474.20095238,
       2726.0952381 , 1417.93333333, 2239.87833333, 2726.0952381 ,
       2682.33214286, 2239.87833333, 3078.08333333, 1417.93333333,
       2239.87833333, 3392.5       , 3086.        , 6704.13333333,
       4399.3       , 4563.7       , 2397.6       , 2177.4       ,
       2790.4       , 2229.73333333, 1693.        , 2590.5       ,
       7645.46571429, 3078.08333333, 2726.0952381 , 3782.75095238,
       1903.3       , 2640.62238095, 7645.46571429, 3392.5       ,
       2239.87833333, 3474.20095238, 2726.0952381 , 1417.93333333,
       2239.87833333, 2726.0952381 , 2682.33214286, 2239.87833333,
       3078.08333333, 1417.93333333, 2239.87833333, 3392.5       ,
       3086.        , 6704.13333333, 4399.3       , 4563.7       ,
       2397.6       , 2177.4       , 2790.4       , 2229.73333333,
       1693.        , 2590.5       , 7645.46571429, 3078.08333

# Annual Prediction result

In [30]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred
mission['Year'] = 2022
pd.set_option('display.max_rows', df.shape[0]+1)
mission


,Name,sales,Year
28,Kamias,2726.095238,2022
29,Kalamasi,3782.750952,2022
30,Papaya,1903.300000,2022
31,Tundan,2640.622381,2022
32,Lakatan,7645.465714,2022
33,Rambutan,3392.500000,2022
34,Suha,2239.878333,2022
35,Atis,3474.200952,2022
36,Guyabano,2726.095238,2022
37,Lemon,1417.933333,2022


# Quarterly Prediction

In [31]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 12
mission['Quarterly'] = 3
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,Quarterly
28,Kamias,227.174603,3
29,Kalamasi,315.229246,3
30,Papaya,158.608333,3
31,Tundan,220.051865,3
32,Lakatan,637.122143,3
33,Rambutan,282.708333,3
34,Suha,186.656528,3
35,Atis,289.516746,3
36,Guyabano,227.174603,3
37,Lemon,118.161111,3


# Monthly Prediction

In [32]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 36
mission['monthly'] = 1
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,monthly
28,Kamias,75.724868,1
29,Kalamasi,105.076415,1
30,Papaya,52.869444,1
31,Tundan,73.350622,1
32,Lakatan,212.374048,1
33,Rambutan,94.236111,1
34,Suha,62.218843,1
35,Atis,96.505582,1
36,Guyabano,75.724868,1
37,Lemon,39.387037,1
